# Get invoice **status transitions**
This endpoint is used to **get** the invoice status transitions occured in archive.

### Endpoint steps
1. Get a token from your credentials by calling the endpoint **_Account/getToken_**;
2. Get an invoice status transitions in archive calling the endpoint **_OutboundFinancialDocument/statusTransitions/{documentId}_**;

#### Response structure from server
When a request is well formed and the authentication data is correct the system responds with a message envelope as follows: 

```Javascript
{
	"CorrelationId": "<GUID>", /* for correlation purposes */
	"IsValid": true,           /* false in case of erros */
	"Errors": [],              /* if empty is a good signal */
	"Data": "<Service Response Data>"   /* the data retuned ex: token, invoice status, dependent on the endpoint called */
}
```

#### Endpoint Server Base URL

In [1]:
# SANDBOX - Test Environment
server_base_adress = "dcn-solution.saphety.com/Dcn.Sandbox.WebApi"

# Saphety Invoice Network - Production Environment
#server_base_adress = "dcn-solution.saphety.com/Dcn.Business.WebApi"

## 1. Get a token (Account/getToken)
Get a token from your account credentials using endpoint api/Account/getToken  
Check [here](./get-token) how to obtain an authentication token. 

## 2. Get a Document status transitions by DocumentId (OutboundFinancialDocument/statusTransitions/{documentId})

### 2.1 Request structure
In the endpoint url you need to supply the **documentId** received

#### documentId
- **Description**: You need to supply the id of the invoice.
- **Required**: Yes
- **Type**: string
- **Example**: "3fa85f64-5717-4562-b3fc-2c963f66afa6"

### 2.2 Request example

In [20]:
outboundFinancialDocumentId = "fc5e547d-8537-4e05-97d5-1159c62efd6f"

# SIN endpoint url for retrieving inforfation on invoice previously sent
service_url = "https://" + server_base_adress + "/api/OutboundFinancialDocument/statusTransitions/" + outboundFinancialDocumentId
print (service_url)

https://dcn-solution.saphety.com/Dcn.Sandbox.WebApi/api/OutboundFinancialDocument/statusTransitions/fc5e547d-8537-4e05-97d5-1159c62efd6f


### 2.3 Call the service to get the document's status transitions
**Get** the invoice status transitions by the **Document id** (response “OutboundFinancialDocumentId“) using endpoint _**/api/OutboundFinancialDocument/statusTransitions/:documentId**_

In [21]:
headers = {
    'Authorization': 'bearer ' + token
}
response = requests.request("GET", service_url, headers=headers)

# formating the response to json for visualization purposes only
json_response = json.loads(response.text)
print(json.dumps(json_response["Data"], indent=4))

[
    {
        "Id": "a6a82dfc-652b-4ac7-aaeb-0cb21058f0b9",
        "OutboundFinancialDocumentId": "fc5e547d-8537-4e05-97d5-1159c62efd6f",
        "IntegrationStatus": "Received",
        "PreviousStatus": "Sent",
        "StatusDate": "2021-11-04 11:54:51",
        "Reason": null
    },
    {
        "Id": "5710c37d-6594-4116-9b68-67e5e60da7cb",
        "OutboundFinancialDocumentId": "fc5e547d-8537-4e05-97d5-1159c62efd6f",
        "IntegrationStatus": "Sent",
        "PreviousStatus": "Not_Sent",
        "StatusDate": "2021-11-04 11:54:09",
        "Reason": null
    }
]


The status transitions of an invoice in archive is returned at response "Data.IntegrationStatus" and "Data.PreviousStatus", see the possible values returned:
* **NotIntegrated:** invoice processed, not sent to the receiver for integration.
* **Not_Sent:** invoice not processed, it's a draft not sent yet to the receiver entity.
* **Error:** invoice processed, sent to the receiver, receiver has returned errors.
* **Sent:** invoice processed, sent to the receiver for integration.
* **Received:** invoice processed, sent to receiver, receiver confirmed integration.
* **Rejected:** invoice processed, sent to receiver, receiver rejects integration.
* **NotIntegrated:** When the document is not integrated
* **Paid:** invoice processed, sent to receiver, receiver confirms payment.
* **Returned:** When the document is returned

For invoices that have "IntegrationStatus" as "Rejected" or "Error", is also advisable to look at Status Transition Reason, at response "Data.Reason", to know the reason descriptions. The "Reason" property is a list that lives inside the Status Transition object. This property can be null or filled with an object list with the following properties:  
    
* **Code**: *string*;  
* **Field**: *string*;  
* **Values**: *List\<string\>*  

Here is an exemple of a document that was rejected with the *Reason* property filled:

``` string
[
    {
        "Id": "980e49f6-ec21-426c-b7da-374a55bff460",
        "OutboundFinancialDocumentId": "4d1041f0-8fee-4296-8f2f-150fd9186e21",
        "IntegrationStatus": "Sending",
        "PreviousStatus": "Not_Sent",
        "StatusDate": "2021-11-04 11:52:11",
        "Reason": null
    },
    {
        "Id": "bafba46e-86d6-4196-803a-58aee72a464e",
        "OutboundFinancialDocumentId": "4d1041f0-8fee-4296-8f2f-150fd9186e21",
        "IntegrationStatus": "Rejected",
        "PreviousStatus": "Sending",
        "StatusDate": "2021-11-04 11:55:24",
        "Reason": [
            {
                "Code": "REJECT",
                "Field": null,
                "Values": [
                    "Encomenda [1234] para o fornecedor [98765] não encontrada"
                ]
            }
        ]4d1041f0-8fee-4296-8f2f-150fd9186e21
    }
]
```

An exemple of a document that had errors while processing

``` string
[
    {
        "Id": "976beaa7-caaf-4436-b895-24aa812d134b",
        "OutboundFinancialDocumentId": "854e3987-b7b3-439e-9401-9f803b56d652",
        "IntegrationStatus": "Sending",
        "PreviousStatus": "Not_Sent",
        "StatusDate": "2021-11-04 12:02:47",
        "Reason": null
    },
    {
        "Id": "1203d10d-3ae7-4f7b-a45c-36dc4d7f330d",
        "OutboundFinancialDocumentId": "854e3987-b7b3-439e-9401-9f803b56d652",
        "IntegrationStatus": "Error",
        "PreviousStatus": "Sending",
        "StatusDate": "2021-11-04 12:02:50",
        "Reason": [
            {
                "Code": "ERROR",
                "Field": null,
                "Values": [
                    "O número de encomenda é inválido (deve conter apenas dígitos e respeitar o seguinte formato: XX/YYYYY)"
                ]
            }
        ]
    }
]
```

And an exemple of a document that was accepted

``` string
[
        {
            "Id": "a59f60f1-cd97-409e-bdf4-8c09d989c80c",
            "OutboundFinancialDocumentId": "425f2ab8-976e-449e-bad9-43dbd47e1b46",
            "IntegrationStatus": "Received",
            "PreviousStatus": "Sending",
            "StatusDate": "2021-11-04 03:34:11",
            "Reason": [
                {
                    "Code": "ACCEPTED",
                    "Field": null,
                    "Values": []
                }
            ]
        },
        {
            "Id": "94662da4-0046-4ce8-86a2-ea523ff252bc",
            "OutboundFinancialDocumentId": "425f2ab8-976e-449e-bad9-43dbd47e1b46",
            "IntegrationStatus": "Sending",
            "PreviousStatus": "Not_Sent",
            "StatusDate": "2021-11-04 03:15:39",
            "Reason": null
        },
        {
            "Id": "de9f608b-85b8-428c-9df7-ac1bf69c4d5a",
            "OutboundFinancialDocumentId": "425f2ab8-976e-449e-bad9-43dbd47e1b46",
            "IntegrationStatus": "Received",
            "PreviousStatus": "Received",
            "StatusDate": "2021-11-04 11:35:41",
            "Reason": [
                {
                    "Code": "ACCEPTED",
                    "Field": null,
                    "Values": []
                }
            ]
        }
    ]
```